<a href="https://colab.research.google.com/github/FariusGitHub/DataScience/blob/master/SOUP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import bs4
import requests
import re
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd

# GETTING POSSIBLE LIST FROM INDEED MULTIPLE PAGES FOR TORONTO DATA SCIENTIST
DSW1=[]; DST1=[]; DSS1= []; DSL1= []; DSC1=[]; DSD1=[]; DSE1=[]; 
soup = bs4.BeautifulSoup(requests.get('https://ca.indeed.com/jobs?q=data+scientist&l=Toronto,+ON&start=').text, 'lxml')

print('collecting toronto data scientist jobs at indeed ...')
for l in tqdm(range(0,int(soup.find('div', id='searchCountPages').text\
                          .strip('\n Page jobs').split(' of')[1])//50*50,50)):
    start = l
    soup = bs4.BeautifulSoup(requests.get('https://ca.indeed.com/jobs?q=data+\
            scientist&l=Toronto,+ON&start='+str(start)+'&limit=50').text, 'lxml')

 
    jk=[]; fccid=[]; 
    for i in range(len(soup.select('.title'))):
        j = 0 if soup.select('.date')[i].text.strip()[:2].isalpha() \
            else int(soup.select('.date')[i].text.strip()[:2])
        k=(datetime.today().date()-timedelta((j))).strftime("%x")
        jk += [str(soup.find_all("script", text=re.compile("jobmap"), \
              type="text/javascript")[0]).split('\n\n')[i+3]\
              .split("]= {jk:'")[1].split(",cmpid:'")[0].split("',efccid:")[0]]
        fccid += [str(soup.find_all("script", text=re.compile("jobmap"), \
                type="text/javascript")[0]).split('\n\n')[i+3]\
                .split("]= {jk:'")[1].split(",cmpid:'")[1].split("',num:'")[0]]
        DSW1+=['http://ca.indeed.com/rc/clk?jk='+jk[i]+'&fccid='+fccid[i]+'&vjs=3'] #WEB
        DST1+=[soup.select('.title')[i].a.get('title')]                             #TITLE
        DSS1+=[soup.select('.summary')[i].text.strip('\n')]                         #SUMMARY
        DSL1+=[soup.select('.sjcl')[i](class_='recJobLoc')[0].get('data-rc-loc')]   #LOC
        DSC1+=[soup.select('.sjcl')[i](class_='company')[0].text.strip('\n')]       #COMPANY 
        DSD1+=['>=30days' if j == 30 else k]                                        #DATE 

# GETTING JOB DESC FROM INDIVIDUAL JOB SITE ABOVE
DSE1=[]
print(len(DSW1))
for i in tqdm(range(len(DSW1))):
    try:
        DSE1 += ['<p>'.join(str(bs4.BeautifulSoup(requests.get(DSW1[i])\
        .text, 'lxml').select(".jobsearch-jobDescriptionText")[0])\
        .split('</p>')).lstrip('<div class="jobsearch-jobDescriptionText" id=\
              "jobDescriptionText"><p>')]
    except Exception:
        DSE1 += ['error during downloading']            #DESC

# PREPARING REPORT AND OPEN EXCEL        
df = pd.DataFrame(list(zip(DSD1, DSC1, DST1, DSL1, DSS1, DSE1, DSW1)), 
      columns =['Posted', 'Company', 'Job Title', 'Location', 'Summary', 'Desc', 'url'])

df

  0%|          | 0/5 [00:00<?, ?it/s]

collecting toronto data scientist jobs at indeed ...


  0%|          | 0/211 [00:00<?, ?it/s]

211


100%|██████████| 211/211 [06:04<00:00,  1.73s/it]


,Posted,Company,Job Title,Location,Summary,Desc,url
0,10/29/20,BMO Financial Group,Senior Data Scientist,"Toronto, ON",Works with various data owners to discover and...,Address:\n<p>60 Yonge Street\n<p><p><div><div>...,http://ca.indeed.com/rc/clk?jk=4200caf72ae464d...
1,10/29/20,York Region,Senior Statistical Data Analyst,"Vaughan, ON",Minimum five (5) years experience in statistic...,error during downloading,http://ca.indeed.com/rc/clk?jk=5e97b9fb204cbc9...
2,10/27/20,Veeva Systems,Data Scientist (Remote),"Toronto, ON",As a Data Scientist for the Business Consultin...,"At Veeva, we build enterprise cloud technology...",http://ca.indeed.com/rc/clk?jk=acdc01645d55764...
3,10/29/20,Johnson & Johnson Family of Companies,"Manager, Data and Analytics Centre","Toronto, ON",Consolidate and reconcile annual data requirem...,"Janssen Pharmaceutical, part of the Janssen Ph...",http://ca.indeed.com/rc/clk?jk=cf725c95910bccb...
4,10/02/20,Nestle,Data Scientist Internship,"North York, ON","Explore data, answering questions and finding ...","A little bit about us</b><br/>\nGood Food, Goo...",http://ca.indeed.com/rc/clk?jk=4798eaba2ef9f6d...
...,...,...,...,...,...,...,...
206,10/24/20,Birdseye Solutions,Principal Visual AI Architect,"Toronto, ON",We have a strong track record delivering secur...,"Company Overview:</b><p>\n<p>\nSince 2011, Bir...",http://ca.indeed.com/rc/clk?jk=4d119d0a9be96e1...
207,>=30days,"Advanced Micro Devices, Inc.",Senior Machine Learning Developer - 79976,"Markham, ON","Engineer appropriate datasets, verify data qua...",What you do at AMD changes everything</b><br/>...,http://ca.indeed.com/rc/clk?jk=634dfa68d3c28b8...
208,>=30days,Prophix Software Inc.,Senior Machine Learning Engineer,"Mississauga, ON","Working with a team of data scientists, produc...",Are you are someone who is passionate about cr...,http://ca.indeed.com/rc/clk?jk=26c8142077632cd...
209,>=30days,Magic Leap - Multiple Locations,Principal Deep Learning Researcher/Engineer,"Toronto, ON",Scene Understanding and Semantics: Design and ...,Job Description</b><p>\n<p>\nWe have an exciti...,http://ca.indeed.com/rc/clk?jk=3c5f2f573fc775d...


In [2]:
for i,j in enumerate(DSW1):
  print(i+1, DSC1[i], j, DSE1[i][:30])

1 BMO Financial Group http://ca.indeed.com/rc/clk?jk=4200caf72ae464dd&fccid=3a429036b3725efa&vjs=3 Address:
<p>60 Yonge Street
<p
2 York Region http://ca.indeed.com/rc/clk?jk=5e97b9fb204cbc9d&fccid=f3c54a468b13c9c9&vjs=3 error during downloading
3 Veeva Systems http://ca.indeed.com/rc/clk?jk=acdc01645d55764a&fccid=353eb997fc901045&vjs=3 At Veeva, we build enterprise 
4 Johnson & Johnson Family of Companies http://ca.indeed.com/rc/clk?jk=cf725c95910bccbc&fccid=08849387e791ebc6&vjs=3 Janssen Pharmaceutical, part o
5 Nestle http://ca.indeed.com/rc/clk?jk=4798eaba2ef9f6d7&fccid=0364df7382483402&vjs=3 A little bit about us</b><br/>
6 RBC http://ca.indeed.com/rc/clk?jk=254e55f1de945058&fccid=537b899e30af3338&vjs=3 1 class="jobSectionHeader"><b>
7 QuadReal http://ca.indeed.com/rc/clk?jk=eefa70bfa123e2a3&fccid=f049514aadb99749&vjs=3 About QuadReal</b><p><p><p><p>
8 Clearbanc http://ca.indeed.com/rc/clk?jk=e6b46b2f4744793d&fccid=aa8192f4729fd6cb&vjs=3 Application Deadline:</b> Frid
9 Moneris So